In [1]:
import pandas as pd
import glob
import os

## Stations

In [2]:
stations = pd.read_csv('./posicion.csv', delimiter=';')
stations.head()

,Codigo,Lat,Long,x,y
0,S01,41.6538,-0.876258,676837.98,4613524.38
1,S02,41.6496,-0.884854,676133.56,4613041.96
2,S03,41.6602,-0.871698,677200.12,4614245.63
3,S04,41.6460,-0.900291,674857.72,4612612.12
4,S05,41.6505,-0.904937,674458.88,4613093.96


In [3]:
stations = stations[['Codigo', 'x', 'y']]
stations.head()

,Codigo,x,y
0,S01,676837.98,4613524.38
1,S02,676133.56,4613041.96
2,S03,677200.12,4614245.63
3,S04,674857.72,4612612.12
4,S05,674458.88,4613093.96


In [4]:
# Change columns name
stations.columns = ['code', 'lat', 'lon']
stations.head()

,code,lat,lon
0,S01,676837.98,4613524.38
1,S02,676133.56,4613041.96
2,S03,677200.12,4614245.63
3,S04,674857.72,4612612.12
4,S05,674458.88,4613093.96


In [5]:
import folium
from pyproj import Transformer

In [6]:
# Transform coordinates from ETRS89 to WRS84
transformer = Transformer.from_crs('EPSG:25830', 'EPSG:4326')

zgz_coords = (41.653858, -0.876258)
map = folium.Map(location=zgz_coords, zoom_start=13)

# Iterate over each station
for index, station in stations.iterrows():
    lat, lon = transformer.transform(station['lat'], station['lon'])

    stations.at[index, 'lat'] = lat
    stations.at[index, 'lon'] = lon

    # Add a marker for the point
    folium.Marker([lat, lon], popup=station['code']).add_to(map)

# If running in a Jupyter Notebook, display the map inline
map

## Temperatures

In [7]:
temperature_files = sorted(glob.glob('./S*ayunt.csv'))
temperature_files

['./S01_CMJ_ayunt.csv',
 './S02_AYT_ayunt.csv',
 './S03_EST_ayunt.csv',
 './S04_IJV_ayunt.csv',
 './S05_DEL_ayunt.csv',
 './S06_IAN_ayunt.csv',
 './S07_PAG_ayunt.csv',
 './S08_ACT_ayunt.csv',
 './S09_FNT_ayunt.csv',
 './S10_AEM_ayunt.csv',
 './S11_BOT_ayunt.csv',
 './S12_OLI_ayunt.csv',
 './S13_CAM_ayunt.csv',
 './S14_VAL_ayunt.csv',
 './S15_POL_ayunt.csv',
 './S16_MOV_ayunt.csv',
 './S17_COG_ayunt.csv',
 './S18_SGR_ayunt.csv',
 './S19_RZD_ayunt.csv',
 './S20_PAT_ayunt.csv',
 './S21_VET_ayunt.csv']

In [8]:
all_dataframes = [pd.read_csv(csv_file, delimiter=',') for csv_file in temperature_files]

# Add a column 'station' with the index of the dataframe
for i, df in enumerate(all_dataframes):
    df['station'] = i+1
    
temperatures = pd.concat(all_dataframes, ignore_index=True)

In [9]:
temperatures.head()

,date,Temp,Hum,station
0,2020-01-01 00:00:00,3.00,84.68,1
1,2020-01-01 01:00:00,3.13,85.13,1
2,2020-01-01 02:00:00,2.48,86.59,1
3,2020-01-01 03:00:00,2.23,87.65,1
4,2020-01-01 04:00:00,1.97,88.40,1


In [10]:
# Split the 'date' column into 'date' and 'time' columns
temperatures[['date', 'time']] = temperatures['date'].str.split(' ', expand=True)
temperatures.head()

,date,Temp,Hum,station,time
0,2020-01-01,3.00,84.68,1,00:00:00
1,2020-01-01,3.13,85.13,1,01:00:00
2,2020-01-01,2.48,86.59,1,02:00:00
3,2020-01-01,2.23,87.65,1,03:00:00
4,2020-01-01,1.97,88.40,1,04:00:00


In [11]:
temperatures = temperatures[temperatures['time'] == '12:00:00']
temperatures.head()

,date,Temp,Hum,station,time
12,2020-01-01,2.40,87.40,1,12:00:00
36,2020-01-02,1.88,87.68,1,12:00:00
60,2020-01-03,3.22,83.81,1,12:00:00
84,2020-01-04,3.82,91.70,1,12:00:00
108,2020-01-05,7.25,76.73,1,12:00:00


In [12]:
dates = [
    "2023-06-02",
    "2023-06-14",
    "2023-06-17",
    "2023-06-22",
    "2023-06-24",
    "2023-06-27",
    "2023-06-29",
    "2023-07-02",
    "2023-07-04",
    "2023-07-09",
    "2023-07-14",
    "2023-07-17",
    "2023-07-19",
    "2023-07-24",
    "2023-07-27",
    "2023-07-29",
    "2023-08-01",
    "2023-08-06",
    "2023-08-08",
    "2023-08-11",
    "2023-08-13",
    "2023-08-16",
    "2023-08-21",
    "2023-08-23"
]

In [13]:
# Filter by date
temperatures = temperatures[temperatures['date'].isin(dates)]
temperatures.head()

,date,Temp,Hum,station,time
29960,2023-06-02,24.58,48.61,1,12:00:00
30248,2023-06-14,26.77,47.33,1,12:00:00
30320,2023-06-17,32.47,35.60,1,12:00:00
30440,2023-06-22,26.42,56.48,1,12:00:00
30488,2023-06-24,29.90,34.76,1,12:00:00


In [14]:
temperatures = temperatures[['date', 'station', 'Temp']]
temperatures.head()

,date,station,Temp
29960,2023-06-02,1,24.58
30248,2023-06-14,1,26.77
30320,2023-06-17,1,32.47
30440,2023-06-22,1,26.42
30488,2023-06-24,1,29.90


In [15]:
# Sort the dataframe by the 'date' column
temperatures = temperatures.sort_values(by=['date', 'station'])

# Reset index
temperatures = temperatures.reset_index(drop=True)

temperatures.head()

,date,station,Temp
0,2023-06-02,1,24.580
1,2023-06-02,2,25.090
2,2023-06-02,3,25.270
3,2023-06-02,4,25.404
4,2023-06-02,5,25.650
